In [1]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
import ROOT

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Welcome to JupyROOT 6.18/02


In [2]:
from tensorflow.keras.layers import LSTM, Bidirectional, Masking, TimeDistributed, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, Activation, Dense, Convolution2D, MaxPooling2D, Dropout, Flatten

In [3]:
tf.enable_eager_execution()

In [4]:
ROOT.gInterpreter.Declare('''
using LorentzVectorM = ROOT::Math::LorentzVector<ROOT::Math::PtEtaPhiM4D<float> >;
using LorentzVectorE = ROOT::Math::LorentzVector<ROOT::Math::PtEtaPhiE4D<float> >;
using LorentzVector = ROOT::Math::LorentzVector<ROOT::Math::PxPyPzE4D<float> >;

ROOT::VecOps::RVec<LorentzVectorE> jets_p4_good(const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{
    ROOT::VecOps::RVec<LorentzVectorE> selected_jets;
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index){
        if(jets_p4.at(jet_index).pt() > 20 && abs(jets_p4.at(jet_index).eta()) < 2.4)
            selected_jets.push_back(jets_p4.at(jet_index));   
    }
    return selected_jets;
}

ROOT::VecOps::RVec<float> jets_p4_pt(const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{   
    ROOT::VecOps::RVec<float> pt(jets_p4.size());
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index)
        pt.push_back(jets_p4.at(jet_index).pt());
    return pt;
}

ROOT::VecOps::RVec<float> jets_p4_eta(const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{   
    ROOT::VecOps::RVec<float> eta(jets_p4.size());
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index)
        eta.push_back(jets_p4.at(jet_index).eta());
    return eta;
}


ROOT::VecOps::RVec<float> jets_p4_E(const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{   
    ROOT::VecOps::RVec<float> E(jets_p4.size());
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index)
        E.push_back(jets_p4.at(jet_index).E());
    return E;
}

ROOT::VecOps::RVec<float> jets_p4_M(const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{   
    ROOT::VecOps::RVec<float> M(jets_p4.size());
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index)
        M.push_back(jets_p4.at(jet_index).M());
    return M;
}



vector<size_t>& good_jets_indexes(const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{    
    static vector<size_t> good_jets_index;
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index){
        if(jets_p4.at(jet_index).pt() > 20 && abs(jets_p4.at(jet_index).eta()) < 2.4)
            good_jets_index.push_back(jet_index);   
    }
    return good_jets_index;

}

ROOT::VecOps::RVec<float> jets_deepFlavour (const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4, const ROOT::VecOps::RVec<float>& jets_deepFlavour_b,
                                const ROOT::VecOps::RVec<float>& jets_deepFlavour_bb, const ROOT::VecOps::RVec<float>& jets_deepFlavour_lepb)
{
    vector<float> jets_deepFlavour;
    for(size_t jet_index = 0; jet_index < jets_p4.size(); ++jet_index){
        if(jets_p4.at(jet_index).pt() > 20 && abs(jets_p4.at(jet_index).eta()) < 2.4){   
            float jet_deepFlavour = jets_deepFlavour_b.at(jet_index) + jets_deepFlavour_bb.at(jet_index)
            + jets_deepFlavour_lepb.at(jet_index);
            jets_deepFlavour.push_back(jet_deepFlavour);
        }
    }
    return jets_deepFlavour;
}

LorentzVectorM getHTTp4(const ROOT::VecOps::RVec<LorentzVectorM>& lep_p4,
                        const ROOT::VecOps::RVec<int>& lep_genTauIndex)
{
    size_t n_tau = 0;
    LorentzVector htt(0, 0, 0, 0);
    for(size_t n = 0; n < lep_p4.size(); ++n) {
        if(lep_genTauIndex.at(n) >= 0) {
            htt += lep_p4.at(n);
            n_tau++;
        }
    }
    if(n_tau != 2)
        throw std::runtime_error("too few taus");
    return LorentzVectorM(htt);
}

ROOT::VecOps::RVec<float> httDeltaPhi(const LorentzVectorM& htt_p4, const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{
    ROOT::VecOps::RVec<float> dphi(jets_p4.size());
    for(size_t n = 0; n < jets_p4.size(); ++n) {
        dphi.at(n) = ROOT::Math::VectorUtil::DeltaPhi(htt_p4, jets_p4.at(n));
    }
    return dphi;
}

ROOT::VecOps::RVec<float> httDeltaEta(const LorentzVectorM& htt_p4, const ROOT::VecOps::RVec<LorentzVectorE>& jets_p4)
{
    ROOT::VecOps::RVec<float> deta(jets_p4.size());
    for(size_t n = 0; n < jets_p4.size(); ++n) {
        deta.at(n) = htt_p4.eta() - jets_p4.at(n).eta();
    }
    return deta;
}

float httMetDeltaPhi(const LorentzVectorM& htt_p4, 
                     const LorentzVectorM& met_p4)
{
    float dphi = ROOT::Math::VectorUtil::DeltaPhi(htt_p4, met_p4);    
    return dphi;
}

float met_pt(LorentzVectorM& met_p4)
{
    return met_p4.pt();
}


''')

True

In [5]:
file_name = '/data/dido/all_samples/GluGluToHHTo2B2Tau_node_SM_eTau_2017_ggHH_NonRes.root'
file_name

'/data/dido/all_samples/GluGluToHHTo2B2Tau_node_SM_eTau_2017_ggHH_NonRes.root'

In [6]:
tauTau_file = file_name
df = ROOT.ROOT.RDataFrame('all_events', file_name)

df = df.Define('jets_p4_good'  , 'jets_p4_good(jets_p4)') \
       .Define('jets_p4_pt' , 'jets_p4_pt(jets_p4_good)') \
       .Define('jets_p4_eta' , 'jets_p4_eta(jets_p4_good)') \
       .Define('jets_p4_E'   , 'jets_p4_E(jets_p4_good)') \
       .Define('jets_p4_M'   , 'jets_p4_M(jets_p4_good)') \
       .Define('htt_p4'      , 'getHTTp4(lep_p4, lep_genTauIndex)') \
       .Define('jets_htt_dphi', 'httDeltaPhi(htt_p4, jets_p4_good)') \
       .Define('jets_htt_deta', 'httDeltaEta(htt_p4, jets_p4_good)') \
       .Define('jets_htt_met_dphi', 'httMetDeltaPhi(htt_p4, pfMET_p4)') \
       .Define('met_pt', 'met_pt(pfMET_p4)') \
       .Define('jets_deepFlavour', 'jets_deepFlavour(jets_p4, jets_deepFlavour_b,  jets_deepFlavour_bb, jets_deepFlavour_lepb)') \
       .Define('n_jets', 'jets_p4_good.size()')
#        .Define('good_jets_indexes', 'good_jets_indexes(jets_p4)') \


In [7]:
df_train = df.Filter('evt % 2 == 0')
df_test = df.Filter('evt % 2 == 1')

In [8]:
evt_columns = [
    'sample_type', 'spin', 'mass_point', 'node', 'sample_year', 'channelId', 'jets_htt_met_dphi', 'met_pt', 'n_jets'
]


jet_columns = [ 'jets_p4_pt', 'jets_p4_eta', 'jets_p4_E', 'jets_p4_M', 'jets_htt_deta', 'jets_deepFlavour', 
               'jets_htt_dphi', 'jets_genJetIndex'
] 

In [9]:
def CreateInputs(raw_data):
    max_jet = int(raw_data['n_jets'].max())
    n_vars = 11 
    n_truth = 6
    evt_var_pos_shift = 1
    jet_var_pos_shift = evt_var_pos_shift + len(evt_columns)
    n_evt = len(raw_data['n_jets'])
    data = np.zeros((n_evt, max_jet, n_truth + n_vars + 1), dtype=np.float32)
    gen_truth = np.zeros((n_evt, max_jet), dtype=np.float32)
    for evt_idx in range(n_evt):
        
        for col_id in range(len(evt_columns)):
            data[evt_idx, :, col_id + evt_var_pos_shift] = raw_data[evt_columns[col_id]][evt_idx]

        for jet_idx in range(int(raw_data['n_jets'][evt_idx])):
            for col_id in range(len(jet_columns) - 1):
                data[evt_idx, jet_idx, col_id + jet_var_pos_shift] = raw_data[jet_columns[col_id]][evt_idx][jet_idx]
            data[evt_idx, jet_idx, len(jet_columns) + jet_var_pos_shift -1] = raw_data[jet_columns[len(jet_columns) -1]][evt_idx][jet_idx] >= 0
    return data   

In [10]:
n_vars = 11 
n_truth = 6
parity = 0 
train_data = None
test_data = None

data_raw_train = df_train.AsNumpy(columns=jet_columns+evt_columns)
data_raw_test = df_test.AsNumpy(columns=jet_columns+evt_columns)

train_data_ch = CreateInputs(data_raw_train)
test_data_ch = CreateInputs(data_raw_test)

if train_data is not None :
    train_data = np.append(train_data, train_data_ch)
else :
    train_data = train_data_ch
    
if test_data is not None :
    test_data = np.append(test_data, test_data_ch)
else :
    test_data = test_data_ch

In [11]:
def sel_acc(y_true, y_pred):
    pred_sorted = tf.argsort(y_pred, axis=1, direction='DESCENDING')
    #return pred_sorted[:, 0]
    n_evt = tf.shape(y_true)[0]
    evt_id = tf.range(n_evt)
    #index_0 = tf.transpose(tf.stack([evt_id, pred_sorted[:, 0]]))
    #index_1 = tf.transpose(tf.stack([evt_id, pred_sorted[:, 1]]))
    index_0 = tf.transpose(tf.stack([evt_id, tf.reshape(pred_sorted[:, 0], shape=(n_evt,))]))
    index_1 = tf.transpose(tf.stack([evt_id, tf.reshape(pred_sorted[:, 1], shape=(n_evt,))]))
    matches_0 = tf.gather_nd(y_true, index_0)
    matches_1 = tf.gather_nd(y_true, index_1)
    valid = tf.cast(tf.equal(matches_0 + matches_1, 2), tf.float32)
    n_valid = tf.reduce_sum(valid)
    return n_valid / tf.cast(n_evt, tf.float32)

In [13]:
def acc_prod(point, year):
    index = []
    for node in range(0, len(point)) :
        index.append((train_data[:, 0, 1] == 7) & (train_data[:, 0, 4] == point[node]) & (train_data[:, 0, 5] == year))
    print(len(index))

    acc = []
    for idx in range(0, len(index)):
        pred = train_data[index[idx], : , n_truth+n_vars:]
        y_pred = train_data[index[idx], :, 15]
        x = sel_acc(pred, y_pred).numpy()
        acc.append(x)
    print(acc)
    return acc

In [ ]:
acc_2016 = acc_prod(nodes_values, 2016)
acc_2017 = acc_prod(nodes_values, 2017)
acc_2018 = acc_prod(nodes_values, 2018)

In [ ]:
# for node in range(len(nodes_label)) :
ax = plt.gca()

df_2017 = pd.DataFrame(data=acc_2017)
df_2018 = pd.DataFrame(data=acc_2018)
    
df_2017.plot(kind='line', label='label',ax=ax)
df_2017.plot(kind='line',ax=ax, color='red', label='label')
plt.gca().legend(('2017','2016'))
plt.yscale('log')

In [ ]:
class HHModel(Model):
    def __init__(self):
        super(HHModel, self).__init__()
        self.lstm_1 = LSTM(8, return_sequences=True)
        self.dropout_1 = Dropout(0.2)
        self.concat_1 = Concatenate()
        self.lstm_2 = LSTM(8, return_sequences=True)
        self.dropout_2 = Dropout(0.2)
        self.dense_1 = TimeDistributed(Dense(10, activation="sigmoid"))
        self.dense_2 = TimeDistributed(Dense(1, activation="sigmoid"))
  
    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.dense_1(x)
        x = self.dropout_1(x)
        x = self.concat_1([inputs, x])
        x = self.lstm_2(x)
        x = self.dropout_2(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        input_shape = tf.shape(inputs)
        return tf.reshape(x, shape=(input_shape[0], input_shape[1]))

In [ ]:
model = HHModel()

In [ ]:
model.call(X_train[:,:,:])

In [ ]:
# Once your model looks good, configure its learning process with .compile():
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[sel_acc])

In [ ]:
model.build(X_train.shape)
model.summary()

In [ ]:
X_train.shape

In [ ]:
Y_train_red = Y_train[:,:,0]
Y_train_red.shape

In [ ]:
model.fit(X_train, Y_train[:,:,0], validation_split=0.2, epochs=1)

In [ ]:
print(X_test.shape)
print(Y_test[:,:,:].shape)

In [ ]:
# evaluate it and print the results:
# test_loss, test_acc = model.evaluate(X_test, Y_test[:,:,0], batch_size=120)

# print('Test accuracy:', test_acc)
# print('Test loss:', test_loss)

In [ ]:
predictions = model.predict(X_test)
predictions[0]

In [ ]:
np.argmax(predictions[0])